In [5]:
# LINEA Analysis Setup - Required Imports and Configuration
import ccxt
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Define exchanges for LINEA analysis
exchanges_list = ['binance', 'bybit']

print("🚀 LINEA Analysis Setup Complete")
print(f"📊 Exchanges configured: {exchanges_list}")
print("✅ Ready for LINEA SPOT vs PERP analysis")

🚀 LINEA Analysis Setup Complete
📊 Exchanges configured: ['binance', 'bybit']
✅ Ready for LINEA SPOT vs PERP analysis


# 🎯 LINEA Analysis - Streamlined Version

## Overview
This notebook provides a simplified analysis of LINEA (SPOT vs PERPETUAL) across multiple exchanges:
- **Binance** (SPOT: `LINEA/USDT`, PERP: `LINEA/USDT:USDT`)
- **Bybit** (SPOT: `LINEA/USDT`, PERP: `LINEA/USDT:USDT`)
- **OKX** (SPOT: `LINEA/USDT`, PERP: `LINEA-USDT-SWAP`)

## Features
✅ **Date Range Filtering** - Specify start and end times  
✅ **Price Comparison** - SPOT vs PERP across exchanges  
✅ **Dollar Volume Analysis** - Volume multiplied by price  
✅ **Clean Visualization** - Interactive charts with Plotly  

## Usage
1. Run cells 1-3 to set up
2. Modify date parameters in cell 4 as needed
3. Run cell 4 to execute analysis and generate visualization

In [6]:
def initialize_exchanges(exchange_names):
    """
    Initialize exchange objects from the given list of exchange names
    """
    exchanges = {}
    
    for exchange_name in exchange_names:
        try:
            if exchange_name.lower() == 'binance':
                exchange = ccxt.binance({
                    'rateLimit': 1200,
                    'enableRateLimit': True,
                })
            elif exchange_name.lower() == 'bybit':
                exchange = ccxt.bybit({
                    'rateLimit': 1200,
                    'enableRateLimit': True,
                })
            elif exchange_name.lower() == 'okx':
                exchange = ccxt.okx({
                    'rateLimit': 1200,
                    'enableRateLimit': True,
                })
            else:
                print(f"Exchange {exchange_name} not supported")
                continue
                
            exchanges[exchange_name] = exchange
            print(f"✓ {exchange_name} initialized successfully")
            
        except Exception as e:
            print(f"✗ Failed to initialize {exchange_name}: {str(e)}")
            
    return exchanges

# Initialize exchanges
exchanges = initialize_exchanges(exchanges_list)
print(f"\nSuccessfully initialized {len(exchanges)} exchanges")

✓ binance initialized successfully
✓ bybit initialized successfully

Successfully initialized 2 exchanges


In [7]:
# Simple LINEA Functions with Date Parameters

def get_LINEA_perp_ticker(exchange_name):
    """Get the correct LINEA perpetual ticker format for each exchange"""
    perp_formats = {
        'binance': 'LINEA/USDT:USDT',
        'bybit': 'LINEA/USDT:USDT', 
        'okx': 'LINEA-USDT-SWAP'
    }
    return perp_formats.get(exchange_name.lower(), 'LINEA/USDT')

def fetch_LINEA_data(ticker, exchanges_list, timeframe='1h', limit=100, 
                   start_time=None, end_time=None):
    """
    Fetch LINEA data for a given ticker across exchanges with date filtering
    
    Parameters:
    - ticker: ticker symbol (e.g., 'LINEA/USDT' or specific perp format)
    - exchanges_list: list of exchange names
    - timeframe: timeframe for data ('1h', '4h', '1d', etc.)
    - limit: number of candles to fetch
    - start_time: start time in 'YYYY-MM-DD HH:MM:SS' format
    - end_time: end time in 'YYYY-MM-DD HH:MM:SS' format
    """
    data = {}
    
    for exchange_name in exchanges_list:
        try:
            exchange = exchanges[exchange_name]
            
            # Convert time parameters if provided
            since = None
            if start_time:
                since = int(datetime.strptime(start_time, '%Y-%m-%d %H:%M:%S').timestamp() * 1000)
            
            print(f"📈 Fetching {ticker} from {exchange_name}...")
            ohlcv = exchange.fetch_ohlcv(ticker, timeframe, since=since, limit=limit)
            
            if ohlcv:
                df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
                df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')
                df.set_index('datetime', inplace=True)
                
                # Filter by end time if provided
                if end_time:
                    end_dt = datetime.strptime(end_time, '%Y-%m-%d %H:%M:%S')
                    df = df[df.index <= end_dt]
                
                # Calculate dollar volume
                df['dollar_volume'] = df['volume'] * df['close']
                
                data[exchange_name] = df
                print(f"  ✅ {exchange_name}: {len(df)} candles, Latest: ${df['close'].iloc[-1]:,.4f}")
            else:
                print(f"  ❌ {exchange_name}: No data")
                
        except Exception as e:
            print(f"  ❌ {exchange_name}: Error - {str(e)}")
    
    return data

def plot_LINEA_simple(spot_data, perp_data, ticker_name="LINEA"):
    """Simple plot showing price and dollar volume for spot and perp"""
    
    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=(f'{ticker_name}: Price Comparison (SPOT vs PERP)', 
                       f'{ticker_name}: Dollar Volume Comparison (SPOT vs PERP)'),
        vertical_spacing=0.12
    )
    
    # Colors: Green for SPOT, Orange for PERP
    spot_colors = ['#2E8B57', '#32CD32', '#90EE90']  
    perp_colors = ['#FF6347', '#FF4500', '#FFB347']  
    
    # Plot SPOT data
    for i, (exchange, df) in enumerate(spot_data.items()):
        color = spot_colors[i % len(spot_colors)]
        
        # Price
        fig.add_trace(
            go.Scatter(
                x=df.index, y=df['close'],
                mode='lines+markers',
                name=f'SPOT {exchange.upper()}',
                line=dict(color=color, width=2.5),
                marker=dict(size=3)
            ), row=1, col=1
        )
        
        # Dollar Volume
        fig.add_trace(
            go.Scatter(
                x=df.index, y=df['dollar_volume'],
                mode='lines',
                name=f'SPOT ${exchange.upper()}',
                line=dict(color=color, dash='dot', width=2),
                showlegend=False
            ), row=2, col=1
        )
    
    # Plot PERP data
    for i, (exchange, df) in enumerate(perp_data.items()):
        color = perp_colors[i % len(perp_colors)]
        
        # Price
        fig.add_trace(
            go.Scatter(
                x=df.index, y=df['close'],
                mode='lines+markers',
                name=f'PERP {exchange.upper()}',
                line=dict(color=color, width=2.5, dash='dash'),
                marker=dict(size=3)
            ), row=1, col=1
        )
        
        # Dollar Volume
        fig.add_trace(
            go.Scatter(
                x=df.index, y=df['dollar_volume'],
                mode='lines',
                name=f'PERP ${exchange.upper()}',
                line=dict(color=color, dash='dashdot', width=2),
                showlegend=False
            ), row=2, col=1
        )
    
    fig.update_layout(
        title=f'{ticker_name} SPOT vs PERP Analysis',
        height=700,
        showlegend=True,
        hovermode='x unified'
    )
    
    fig.update_yaxes(title_text="Price (USD)", row=1, col=1)
    fig.update_yaxes(title_text="Dollar Volume (USD)", row=2, col=1)
    fig.update_xaxes(title_text="Time", row=2, col=1)
    
    fig.show()
    
    # Print summary
    print(f"\n📊 {ticker_name} Summary:")
    if spot_data:
        print("SPOT:")
        for exchange, df in spot_data.items():
            print(f"  {exchange}: ${df['close'].iloc[-1]:,.4f}")
    if perp_data:
        print("PERP:")
        for exchange, df in perp_data.items():
            print(f"  {exchange}: ${df['close'].iloc[-1]:,.4f}")

print("✅ Simple LINEA functions with date parameters loaded!")
print("📝 Use start_time='2025-09-04 10:00:00' and end_time='2025-09-04 12:00:00' format")

✅ Simple LINEA functions with date parameters loaded!
📝 Use start_time='2025-09-04 10:00:00' and end_time='2025-09-04 12:00:00' format


In [8]:
# Simple LINEA Analysis with Date Parameters

# Configuration
TICKER_BASE = "LINEA"
TIMEFRAME = "1h"
LIMIT = 1000  # Increased limit to get more historical data

# Date parameters - Full timeline since LINEA launch
START_TIME = "2025-08-22 00:00:00"  #  launch date
END_TIME = "2025-09-05 23:59:59"    # Current date

print(f"🎯 Complete {TICKER_BASE} Historical Analysis")
print(f"📊 Timeframe: {TIMEFRAME}, Limit: {LIMIT}")
print(f"🏪 Exchanges: {exchanges_list}")
print(f"📅 Full Timeline: {START_TIME} to {END_TIME}")
print(f"⏰ Analysis Period: ~{(datetime.strptime(END_TIME, '%Y-%m-%d %H:%M:%S') - datetime.strptime(START_TIME, '%Y-%m-%d %H:%M:%S')).days} days")
print("="*70)

# Fetch SPOT data
print("\n💰 Fetching COMPLETE SPOT data timeline...")
LINEA_spot = fetch_LINEA_data("LINEA/USDT", exchanges_list, TIMEFRAME, LIMIT, START_TIME, END_TIME)

# Fetch PERP data
print("\n🔄 Fetching COMPLETE PERP data timeline...")
LINEA_perp = {}
for exchange_name in exchanges_list:
    perp_ticker = get_LINEA_perp_ticker(exchange_name)
    perp_data = fetch_LINEA_data(perp_ticker, [exchange_name], TIMEFRAME, LIMIT, START_TIME, END_TIME)
    if perp_data:
        LINEA_perp.update(perp_data)

# Create the plot
if LINEA_spot or LINEA_perp:
    print(f"\n📈 Creating complete {TICKER_BASE} historical visualization...")
    plot_LINEA_simple(LINEA_spot, LINEA_perp, TICKER_BASE)
    
    # Print data summary
    print(f"\n📊 Data Summary:")
    if LINEA_spot:
        for exchange, df in LINEA_spot.items():
            print(f"  SPOT {exchange}: {len(df)} candles from {df.index[0]} to {df.index[-1]}")
    if LINEA_perp:
        for exchange, df in LINEA_perp.items():
            print(f"  PERP {exchange}: {len(df)} candles from {df.index[0]} to {df.index[-1]}")
else:
    print(f"❌ No {TICKER_BASE} data available")

print(f"\n✅ Complete {TICKER_BASE} historical analysis finished!")

🎯 Complete LINEA Historical Analysis
📊 Timeframe: 1h, Limit: 1000
🏪 Exchanges: ['binance', 'bybit']
📅 Full Timeline: 2025-08-22 00:00:00 to 2025-09-05 23:59:59
⏰ Analysis Period: ~14 days

💰 Fetching COMPLETE SPOT data timeline...
📈 Fetching LINEA/USDT from binance...
  ❌ binance: Error - binance does not have market symbol LINEA/USDT
📈 Fetching LINEA/USDT from bybit...
  ❌ bybit: Error - bybit does not have market symbol LINEA/USDT

🔄 Fetching COMPLETE PERP data timeline...
📈 Fetching LINEA/USDT:USDT from binance...
  ✅ binance: 98 candles, Latest: $0.0273
📈 Fetching LINEA/USDT:USDT from bybit...
  ✅ bybit: 96 candles, Latest: $0.0275

📈 Creating complete LINEA historical visualization...



📊 LINEA Summary:
PERP:
  binance: $0.0273
  bybit: $0.0275

📊 Data Summary:
  PERP binance: 98 candles from 2025-09-01 08:00:00 to 2025-09-05 09:00:00
  PERP bybit: 96 candles from 2025-09-01 10:00:00 to 2025-09-05 09:00:00

✅ Complete LINEA historical analysis finished!
